# Wrangle OpenStreetMap Using Python and SQL

--------

### Author: Susan Li
### Date: February 8 2017

## Map Area:
------------
### Boston Massachusetts, United States

* https://www.openstreetmap.org/export#map=11/42.3126/-70.9978

* https://mapzen.com/data/metro-extracts/metro/boston_massachusetts/

This map is of the city of Boston Massachusetts, the reason I chose this map is that we love Boston and visit the city often. I would like the opportunity to make the contribution to its improvement on openstreetmap.org. With Boston Marathon is only a couple of months away, we can't wait to go back.

## Problems Encountered in the Map
---------
After downloading the dataset and running it against a provisional data.py file, I noticed the following main problems with the dataset, as follows.

* Nonuniform and abbreviated street types ('St.', 'St', 'ST', 'Ave', 'Ave.', 'Pkwy', 'Ct', 'Sq', etc).

* Inconsistent postal codes ('MA 02118', '02118-0239')

### Abbrevated street names
This part of the project is to auditing and cleaning nonuniform and abbreviated street types. I will attemp to replace street type "ST, St, St., St, st, street" with "Street", replace "Ave, Ave." with "Avenue", replace "Pkwy" with "ParkWay", replace "Rd, Rd., rd." with "Road", replace "Ct" with "Court", replace "Dr" with "Drive", replace "Hwy" with "Highway", and replace "Sq" with "Sqaure". The following are the steps I will be taking, as showing in audit.py. 

* Create a list of expected street types that do not need to be cleaned.
* The function "audit_street_type" collects the last words in the "street_name" strings, if they are not within the expected list, then stores them in the dictionary "street_types". This will allow me to see the nonuniform and abbreviated street types being used and give me a better sense of the data as a whole.
* The "is_street_name" function looks for tags that specify street names (k="addr:street"). 
* The "audit" function returns a dictionary that match the above function conditions. 
* The update_name function takes an old name to mapping dictionary, and update to a new one.

In [ ]:
def update_name(name, mapping):
    """takes an old name to mapping dictionary, and update to a new one"""
    m = street_type_re.search(name)
    if m not in expected:
        if m.group() in mapping.keys():
            name = re.sub(m.group(), mapping[m.group()], name)
    
    return name

This updated substrings in abbreviated address strings, such that: "Boston St" becomes: "Boston Street".

### Validate Postal Codes
* Boston area postal codes begin with "021, 024 and 022". The formats of the post code are vary, some are five-digits, some are nine-digits, and some start from state characters. 
* Validating postal codes is a bit complicated. However, I decided to drop leading and trailing characters before and after the main 5 digit post code. This dropped the state characters "MA" from "MA 02118", and dropped "0239" from "02118-0239", using the following function in audit_postcode.py.

In [ ]:
def update_postcode(postcode):
    """Clean postcode to a uniform format of 5 digit; Return updated postcode"""
    if re.findall(r'^\d{5}$', postcode): # 5 digits 02118
        valid_postcode = postcode
        return valid_postcode
    elif re.findall(r'(^\d{5})-\d{4}$', postcode): # 9 digits 02118-0239
        valid_postcode = re.findall(r'(^\d{5})-\d{4}$', postcode)[0]
        return valid_postcode
    elif re.findall(r'MA\s*\d{5}', postcode): # with state code MA 02118
        valid_postcode =re.findall(r'\d{5}', postcode)[0]  
        return valid_postcode  
    else:
        return None

## Prepare for Database - SQL
The next step is to prepare the data to be inserted into a SQL database in data.py.

* To do so I will parse the elements in the OSM XML file, transforming them from document format to tabular format, thus making it possible to write to .csv files.  These csv files can then easily be imported to a SQL database as tables.
* The "shape_element" function is used to transform each element in the correct format, the function is passed each individual parent element from the ".osm" file, one by one (when it is called in the "process_map" function).

## The Overview Statistics of The Dataset and Database Queries
This section contains basic statistics about the dataset, the SQL queries used to gather them, and some additional ideas about the data in context.

### The size of each file

In [8]:
import os
print('The boston_massachusetts.osm file is {} MB'.format(os.path.getsize('boston_massachusetts.osm')/1.0e6))
print('The project.db file is {} MB'.format(os.path.getsize('project.db')/1.0e6))
print('The nodes.csv file is {} MB'.format(os.path.getsize('nodes.csv')/1.0e6))
print('The nodes_tags.csv file is {} MB'.format(os.path.getsize('nodes.csv')/1.0e6))
print('The ways.csv file is {} MB'.format(os.path.getsize('ways.csv')/1.0e6))
print('The ways_tags.csv is {} MB'.format(os.path.getsize('ways_tags.csv')/1.0e6))
print('The ways_nodes.csv is {} MB'.format(os.path.getsize('ways_nodes.csv')/1.0e6)) # Convert from bytes to MB

The boston_massachusetts.osm file is 434.233159 MB
The project.db file is 135.95136 MB
The nodes.csv file is 46.200929 MB
The nodes_tags.csv file is 46.200929 MB
The ways.csv file is 22.793719 MB
The ways_tags.csv is 21.643234 MB
The ways_nodes.csv is 13.488694 MB


### Create SQL Database and Tables

In [2]:
import sqlite3
import csv
from pprint import pprint

sqlite_file = "project.db"
conn = sqlite3.connect(sqlite_file)
cur = conn.cursor()

cur.execute('DROP TABLE IF EXISTS nodes')
conn.commit()

cur.execute('''
    Create Table nodes(id INTEGER, lat REAL, lon REAL, user TEXT, uid INTEGER, version TEXT, changeset INTEGER, timestamp TEXT)
''')

conn.commit()

In [3]:
with open('nodes.csv','r', encoding = 'utf-8') as fin:
    dr = csv.DictReader(fin)
    to_db = [(i['id'], i['lat'], i['lon'], i['user'], i['uid'], i['version'], i['changeset'], i['timestamp']) for i in dr]
    cur.executemany("INSERT INTO nodes(id, lat, lon, user, uid, version, changeset, timestamp) VALUES (?,?,?,?,?,?,?,?);", to_db)

conn.commit()

### Number of nodes

In [9]:
cur.execute("SELECT COUNT(*) FROM nodes;")
print(cur.fetchall())

[(1048574,)]


### Number of ways

In [15]:
cur.execute("SELECT COUNT(*) FROM ways;")
print(cur.fetchall())

[(618026,)]


### Number of unique users

In [16]:
cur.execute("SELECT COUNT(DISTINCT(e.uid)) FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) e;")
print(cur.fetchall())

[(922,)]


### Top 10 contributing users

In [17]:
import pprint
cur.execute("SELECT e.user, COUNT(*) as num FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e GROUP BY e.user ORDER BY num DESC LIMIT 10;")
pprint.pprint(cur.fetchall())

[('', 833300),
 ('crschmidt', 402572),
 ('jremillard-massgis', 231863),
 ('MassGIS Import', 63279),
 ('wambag', 30943),
 ('ryebread', 20247),
 ('OceanVortex', 17772),
 ('Ahlzen', 5092),
 ('ingalls_imports', 4220),
 ('morganwahl', 3740)]


### Number of Starbucks

In [21]:
cur.execute("SELECT COUNT(*) FROM nodes_tags WHERE value LIKE '%Starbucks%';")
print(cur.fetchall())

[(55,)]


### Count Tourism Related Categories Descending

In [22]:
import pprint
cur.execute ("SELECT tags.value, COUNT(*) as count FROM (SELECT * FROM nodes_tags UNION ALL \
             SELECT * FROM ways_tags) tags \
             WHERE tags.key LIKE '%tourism'\
             GROUP BY tags.value \
             ORDER BY count DESC;")
pprint.pprint(cur.fetchall())

[('hotel', 87),
 ('museum', 54),
 ('artwork', 48),
 ('attraction', 33),
 ('viewpoint', 30),
 ('picnic_site', 26),
 ('information', 25),
 ('guest_house', 8),
 ('hostel', 4),
 ('motel', 3),
 ('aquarium', 2),
 ('chalet', 2),
 ('zoo', 2),
 ('gallery', 1),
 ('theme_park', 1)]


## Further Data Exploration

### Number of Restaurants in each city descending

In [23]:
import pprint
cur.execute("SELECT nodes_tags.value, COUNT(*) as num FROM nodes_tags JOIN (SELECT DISTINCT(id) \
            FROM nodes_tags WHERE value = 'restaurant') i ON nodes_tags.id = i.id WHERE nodes_tags.key = 'city'\
            GROUP BY nodes_tags.value ORDER BY num DESC;")

pprint.pprint(cur.fetchall())

[('Cambridge', 51),
 ('Boston', 44),
 ('Somerville', 18),
 ('Brookline', 10),
 ('East Boston', 5),
 ('Brookline, MA', 4),
 ('Arlington', 3),
 ('Medford', 3),
 ('Watertown', 3),
 ('Allston', 2),
 ('Arlington. MA', 2),
 ('Charlestown', 2),
 ('Chestnut Hill', 2),
 ('Jamaica Plain', 2),
 ('Malden', 2),
 ('2067 Massachusetts Avenue', 1),
 ('Arlington, MA', 1),
 ('Boston, MA', 1),
 ('Cambridge, MA', 1),
 ('Cambridge, Massachusetts', 1),
 ('Chelsea', 1),
 ('Everett', 1),
 ('Quincy', 1),
 ('Watertown, MA', 1),
 ('boston', 1),
 ('somerville', 1),
 ('winthrop', 1)]


### Top 5 Most Popular Fast Food Chain

In [24]:
cur.execute ("SELECT nodes_tags.value, COUNT(*) as num FROM nodes_tags JOIN (SELECT DISTINCT(id) \
             FROM nodes_tags WHERE value='fast_food') i ON nodes_tags.id=i.id WHERE nodes_tags.key='name' \
             GROUP BY nodes_tags.value ORDER BY num DESC LIMIT 5;")

pprint.pprint(cur.fetchall())

[("Dunkin' Donuts", 12),
 ('Subway', 11),
 ("McDonald's", 8),
 ('Burger King', 7),
 ("Wendy's", 5)]


### Top 5 Cafe Chain

In [25]:
cur.execute ("SELECT nodes_tags.value, COUNT(*) as num FROM nodes_tags JOIN (SELECT DISTINCT(id) \
             FROM nodes_tags WHERE value='cafe') i ON nodes_tags.id=i.id WHERE nodes_tags.key='name' \
             GROUP BY nodes_tags.value ORDER BY num DESC LIMIT 5;")

pprint.pprint(cur.fetchall())

[("Dunkin' Donuts", 41),
 ('Starbucks', 41),
 ('Au Bon Pain', 7),
 ("Peet's Coffee", 5),
 ('Starbucks Coffee', 5)]


## Additional Ideas

After the above review and analysis,, it is obvious that the Boston area is not complete and accurate, though I believe street names and post codes have been cleaned for the purpose of this exercise. For example from above queries, "Starbucks" appeared twice in the same list and "Dunkin Donuts" is both fast food restaurant and cafe. A more standardized data entry should be implemented.

In addition, I noticed that there are significant street address data and postcodes data missing when I was updating street names postcodes at the start of the project. Validating data is an important part of OpenStreetMap, having experienced volunteers check OSM data to make sure it is complete, accurate and thorough by using [Quality Assurance tools](http://wiki.openstreetmap.org/wiki/Quality_assurance).

Also from the above review, there are only 80 hotels in Boston. A quick search on [booking.com](http://www.booking.com/city/us/boston.html) gives me a result of 304 properties. I think OpenStreetMap should encourage hotels, restaurants and other local business owners to publish on OSM, and edit their own content to make it more accurate and make it available for everyone to benefit from.

## Benefits and Anticipated Problems in Implementing the Improvement

### Benefits

* Local people search maps to look for amenities and tourists search maps to look for interesting attractions and popular restaurants. There are potential to extend OpenStreetMap to include travel, walking and biking tour information, user reviews of establishments, local public transit etc. If more people use it, more people adding data, and this makes the maps better for everyone. This could expand OpenStreetMap's user base, and become more consumer focused.

* Parks, schools, hotels, restaurants and other local businesses should be able to update and edit their own data on OSM, to ensure the accuracy and completeness.

* If more businesses tap OSM for its own service, and edit content to make it more detailed, precise and accurate, this will also make OSM community more diverse.

### Anticipated problems

* As we all aware, OpenStreeMap has no paid employees, no premises. The missing post codes problem lies in not being able to convert a post code to a latitude/longitude figure, and then back again. The data to easily enable this isn’t available in many countries. And it's easier to figure out street names by walking around, but not post codes.

* From an end-user's perspective, how many people have ever used OpenStreetMap? Or even heard about it? Can anyone relies on OSM as much as slick Google app? If there is not enough awareness, how could we expect local businesses to pay attention to it?

* Because there is no cost to use the data, if a company taps OSM for its own service such as a mobile app. It is hard for OSM to get the attribution it deserves. According to OSM foundation chairman Simon Poole, [lack of attribution is a major issue](http://www.openstreetmap.org/user/SimonPoole/diary/20719).

Notebook created by: [Susan Li](https://www.linkedin.com/in/susanli/)